In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
import xarray as xr
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

warnings.filterwarnings("ignore",message='invalid value encountered in less_equal')

%matplotlib inline

In [2]:
savePath = '/Users/danstechman/GoogleDrive/School/Research/PECAN/Microphysics/plots/vertical_profiles'
fType = 'pdf'

noDispSave = True


flights = ['20150617','20150620','20150701','20150702','20150706','20150709']
# flights = ['20150620']

In [3]:
sTot = 42
ntChange = np.zeros((sTot,1))
twcChange = np.zeros((sTot,1))
tempLapse = np.zeros((sTot,1))
iSprl = 0
for flight in flights:
    print('Working on {}...'.format(flight))

    cipFile = '/Users/danstechman/GoogleDrive/PECAN-Data/mp-data/' + flight + '/' + flight + '_CIPfit-spirals-10s1sAvg.nc'
    pecanPrmF = '/Users/danstechman/GoogleDrive/PECAN-Data/' + flight + '_PECANparams.nc'
    flFile = '/Users/danstechman/GoogleDrive/PECAN-Data/FlightLevelData/Processed/' + flight + '_FltLvl_Processed.nc'
    
    # Pull out any PECAN parameters
    pecanPrms = xr.open_dataset(pecanPrmF,decode_times=False)
    mlBotTemp = pecanPrms.mlBotTemp.data
    mlTopTemp = pecanPrms.mlTopTemp.data
    startT = pecanPrms.startT.data
    endT = pecanPrms.endT.data
    
    # Pull out FL data
    flData = xr.open_dataset(flFile,decode_times=False)
    timeSecs_FL = flData.time_secs_FL.data
    fl_tempC = flData.TA.data
    fl_rh = flData.RH_hybrid.data
    fl_alt_mMSL = flData.Alt.data
    fl_windSpd_ms = flData.windSpd.data

    # Pull out any global variables/attributes from the netcdf file
    cipData_root = xr.open_dataset(cipFile)
    sprlZone = str(cipData_root.sprlZone.data,'utf-8')
    mcsType = str(cipData_root.mcsType.data,'utf-8')
    numSprls = len(sprlZone)
    
    
    # Loop over each spiral for the current flight
    for ix in np.arange(0,numSprls):
#     for ix in np.arange(3,4):
        print('\tAnalyzing Spiral {}'.format(ix+1))
        
         # Get start and end indices for FL variables within current spiral
        strtMatch = min(timeSecs_FL, key=lambda x: abs(x - startT[ix]))
        endMatch = min(timeSecs_FL, key=lambda x: abs(x - endT[ix]))
        flStrtIx = np.squeeze(np.where(timeSecs_FL == strtMatch))
        flEndIx = np.squeeze(np.where(timeSecs_FL == endMatch))
        
        flRHsprl = fl_rh[flStrtIx:flEndIx]
        flAltSprl = fl_alt_mMSL[flStrtIx:flEndIx]
        flTempCsprl = fl_tempC[flStrtIx:flEndIx]
        
        # Open the group associated with the current spiral
        cipData = xr.open_dataset(cipFile,group='spiral_' + str(ix+1))

        Nt = cipData.cipNt_hybrid_igf.data
        twc = cipData.cipTWC_hybrid_igf_mlr.data
        tempC_10s = cipData.tempC_10s.data
        cipTime_10s = cipData.cipTsec_10s.data
        
        twc[twc == 0] = np.nan
        Nt[Nt == 0] = np.nan
        
        # Get index of temp nearest 0 deg C for CIP variables
        negTmatch_cip = min(tempC_10s, key=lambda x: abs(x - 0.0))
        negTmatch_cipIx = np.squeeze(np.where(tempC_10s == negTmatch_cip))
        if tempC_10s[0] > tempC_10s[-1]:
            sprlUp = True
            cipTempCsprl = tempC_10s[negTmatch_cipIx:]
            cipNtSprl = Nt[negTmatch_cipIx:]
            cipTWCsprl = twc[negTmatch_cipIx:]
            
#             tempLapse[iSprl] = (flTempCsprl[-1] - flTempCsprl[0])/((flAltSprl[-1] - flAltSprl[0])/1000)
        else:
            sprlUp = False
            cipTempCsprl = tempC_10s[:negTmatch_cipIx]
            cipNtSprl = Nt[:negTmatch_cipIx]
            cipTWCsprl = twc[:negTmatch_cipIx]
            
#             tempLapse[iSprl] = (flTempCsprl[0] - flTempCsprl[-1])/((flAltSprl[0] - flAltSprl[-1])/1000)
        

        
       
        
        
        
            
        finiteIx = np.isfinite(np.log10(cipNtSprl)) & np.isfinite(cipTempCsprl)
        m_Nt, b_Nt = np.polyfit(np.log10(cipNtSprl[finiteIx]), cipTempCsprl[finiteIx], 1)
        NtFit = m_Nt*np.log10(cipNtSprl) + b_Nt
        ntChange[iSprl] = 1 - ((10**((-5 - b_Nt)/m_Nt))/(10**((-6 - b_Nt)/m_Nt))) # 1 deg C change percentage
        
        finiteIx = np.isfinite(np.log10(cipTWCsprl)) & np.isfinite(cipTempCsprl)
        m_twc, b_twc = np.polyfit(np.log10(cipTWCsprl[finiteIx]), cipTempCsprl[finiteIx], 1)
        twcFit = m_twc*np.log10(cipTWCsprl) + b_twc
        twcChange[iSprl] = 1 - ((10**((-5 - b_twc)/m_twc))/(10**((-6 - b_twc)/m_twc))) # 1 deg C change percentage
        
        finiteIx = np.isfinite(flAltSprl) & np.isfinite(flTempCsprl)
        m_temp, b_temp = np.polyfit(flTempCsprl[finiteIx], flAltSprl[finiteIx]/1000, 1)
        tempFit = m_temp*flTempCsprl + b_temp
        tempLapse[iSprl] = ((6 - b_temp)/m_temp) - ((5 - b_temp)/m_temp)
        
        iSprl += 1

Working on 20150617...
	Analyzing Spiral 1
	Analyzing Spiral 2
	Analyzing Spiral 3
	Analyzing Spiral 4
	Analyzing Spiral 5
	Analyzing Spiral 6
	Analyzing Spiral 7
Working on 20150620...
	Analyzing Spiral 1
	Analyzing Spiral 2
	Analyzing Spiral 3
	Analyzing Spiral 4
	Analyzing Spiral 5
	Analyzing Spiral 6
	Analyzing Spiral 7
Working on 20150701...
	Analyzing Spiral 1
Working on 20150702...
	Analyzing Spiral 1
	Analyzing Spiral 2
	Analyzing Spiral 3
Working on 20150706...
	Analyzing Spiral 1
	Analyzing Spiral 2
	Analyzing Spiral 3
	Analyzing Spiral 4
	Analyzing Spiral 5
	Analyzing Spiral 6
	Analyzing Spiral 7
	Analyzing Spiral 8
Working on 20150709...
	Analyzing Spiral 1
	Analyzing Spiral 2
	Analyzing Spiral 3
	Analyzing Spiral 4
	Analyzing Spiral 5
	Analyzing Spiral 6
	Analyzing Spiral 7
	Analyzing Spiral 8
	Analyzing Spiral 9
	Analyzing Spiral 10
	Analyzing Spiral 11
	Analyzing Spiral 12
	Analyzing Spiral 13
	Analyzing Spiral 14
	Analyzing Spiral 15
	Analyzing Spiral 16


In [4]:
np.savetxt('/Users/danstechman/Desktop/tempLapse.csv', tempLapse, delimiter=',')

In [ ]:
fig, ax = plt.subplots(figsize=(16,20))

                
ax.axhline(y=mlTopTemp[ix],color='black',linestyle='--',linewidth=3,label='ML Top')
ax.axhline(y=mlBotTemp[ix],color='red',linestyle='--',linewidth=3,label='ML Bottom')
ax.plot(np.log10(cipNtSprl),cipTempCsprl,'bo',markeredgecolor='white',markeredgewidth=1,markersize=12)
ax.plot(np.log10(cipNtSprl),NtFit,'r-')

ax.set_ylim(-18.5,22)
# ax.set_xscale('log',nonposx='mask')
ax.invert_yaxis()
ax.set_ylabel('Temperature ($^{\circ}$C)',fontsize=24)
ax.tick_params(axis='both', which='major', labelsize=22)
ax.grid(which='both')
ax.legend(loc='lower left',fontsize=18)